In [0]:
import pickle as pkl
import glob
import time
import os
import numpy as np
import sys
from music21 import converter, instrument, note, stream, chord, duration

import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape, Bidirectional
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop



In [75]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#sys.path.append('/content/gdrive/My Drive/cs230')
project_folder = "/content/gdrive/My Drive/cs230"
#data_folder = project_folder + "/data"
#jazz_folder = project_folder + "/data/sumuzhao/Jazz"
#classical_folder = project_folder + "/data/sumuzhao/Classic"
#testing_folder = project_folder + "/data/testing"
#pop_folder = project_folder + "/data/pop/examples"
pop_folder = project_folder + "/data/pop/supreme"

In [0]:
section = 'compose'
run_id = 'supreme'
music_name = 'pop'
run_folder = project_folder + '/run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

# model params
embed_size = 100
rnn_units = 256

In [0]:
store_folder = os.path.join(run_folder, 'store')

with open(os.path.join(store_folder, 'distincts'), 'rb') as filepath:
    distincts = pkl.load(filepath)
    note_names, n_notes, duration_names, n_durations = distincts

with open(os.path.join(store_folder, 'lookups'), 'rb') as filepath:
    lookups = pkl.load(filepath)
    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups

In [79]:
len(note_names), n_notes, len(duration_names), n_durations

(2299, 2299, 17, 17)

In [0]:
def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in) 

    x = Concatenate()([x1,x2])
    x = Bidirectional(LSTM(rnn_units, return_sequences=True))(x)
    x = LSTM(rnn_units, return_sequences=True)(x)

    e = Dense(1, activation='tanh')(x)
    e = Reshape([-1])(e)
    alpha = Activation('softmax')(e)
    alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

    c = Multiply()([x, alpha_repeated])
    c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
                                    
    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
    durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
    model = Model([notes_in, durations_in], [notes_out, durations_out])

    opti = RMSprop(lr = 0.001)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)

    return model

In [81]:
weights_folder = os.path.join(run_folder, 'weights')
weights_file = 'weights.h5'

model = create_network(n_notes, n_durations, embed_size, rnn_units)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 100)    229900      input_9[0][0]                    
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 100)    1700        input_10[0][0]                   
____________________________________________________________________________________________

In [0]:
def sample_with_temp(preds, temperature):

    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [0]:
# prediction params
notes_temp=0.5
duration_temp = 0.5
max_extra_notes = 32
max_seq_len = 32
seq_len = 32

In [0]:
def load_notes_durations_training():
  # use this to sample a sequence from the whole training set to seed music generation

  # load notes of all the training files
  with open(os.path.join(store_folder, 'notes'), 'rb') as f:
    notes = pkl.load(f)
  
  with open(os.path.join(store_folder, 'durations'), 'rb') as f:
    durations = pkl.load(f)

  return notes, durations
# notes = ['START']
# durations = [0]

# if seq_len is not None:
#     notes = ['START'] * (seq_len - len(notes)) + notes
#     durations = [0] * (seq_len - len(durations)) + durations

In [0]:
def load_notes_durations_song(fname, folder):
# use this to sample a sequence for a particular song to seed music generation"
    music_file = glob.glob(os.path.join(folder, fname))[0]

    notes = []
    durations = []

    # These dictionaries contain the number half steps required to transpose music to either C major or A minor
    majors = dict([('A-', 4),('G#', 4),('A', 3),('A#', 2),('B-', 2),('B', 1),('C', 0),('C#', -1),('D-', -1),('D', -2),('D#', -3),('E-', -3),('E', -4),('F', -5),('F#', 6),('G-', 6),('G', 5)])
    minors = dict([('G#', 1), ('A-', 1),('A', 0),('A#', -1),('B-', -1),('B', -2),('C', -3),('C#', -4),('D-', -4),('D', -5),('D#', 6),('E-', 6),('E', 5),('F', 4),('F#', 3),('G-', 3),('G', 2)])

    original_score = converter.parse(music_file).chordify()
        
    key = original_score.analyze('key')
      
    if key.mode == "major":
        halfSteps = majors[key.tonic.name]
    elif key.mode == "minor":
        halfSteps = minors[key.tonic.name]

    score = original_score.transpose(halfSteps)
   
    for element in score.flat:
        if isinstance(element, note.Note):
            if element.isRest:
                notes.append(str(element.name))
                durations.append(element.duration.quarterLength)
            else:
                notes.append(str(element.nameWithOctave))
                durations.append(element.duration.quarterLength)

        if isinstance(element, chord.Chord):
            notes.append('.'.join(n.nameWithOctave for n in element.pitches))
            durations.append(element.duration.quarterLength)

    return notes, durations

In [0]:
#seed_notes, seed_durations = load_notes_durations_training()
seed_notes, seed_durations = load_notes_durations_song("crazy love.mid", pop_folder)


In [88]:
seed_notes

['F3',
 'F3.C4',
 'F3.G3.C4',
 'F3.A3.C4',
 'F3.A3.C4',
 'D3.A3.C4',
 'D3.G3.C4',
 'E3.G3.C4',
 'D3.G3.C4',
 'C3.D3.G3.C4',
 'C2.C3.D3.G3.C4',
 'C2.C3.E3.G3.C4',
 'C2.C3.E3.G3.C4.E4',
 'C2.C3.E3.G3.C4.E4',
 'C2.C3.E3.G3.C4.F4',
 'C2.E3.G4',
 'C2.E3.G4',
 'C2.E3.G4',
 'F2.F3.G4',
 'F2.F3.C4.G4',
 'F2.F3.A3.C4.G4',
 'F2.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'F2.F3.A3.C4.E4.A4',
 'F2.F3.A3.C4.E4.A4',
 'F2.F3.A3.C4.E4.A4',
 'F2.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'G2.F3.A3.C4.E4.B4',
 'G2.F3.A3.B3.E4.B4',
 'G2.F3.G3.B3.E4.B4',
 'G2.F3.G3.B3.G4.B4',
 'G2.D3.G3.B3.G4.B4',
 'G2.D3.G3.B3.G4.C5',
 'G2.D3.G3.B3.G4.C5',
 'G2.D3.G3.B3.G4.C5',
 'F2.G2.F3.G3.B3.G4.A4',
 'F2.G2.F3.G3.C4.G4.A4',
 'F2.G2.F3.A3.C4.G4.A4',
 'F2.G2.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.A4',
 'C3.F3.A3.C4.E4.B4',
 'C3.F3.A3.C4.E4.B4',
 'C3.F3.A3.C4.E4.B4',
 'C2.C3.F3.A3.C4.E4.B4',
 'C2.C3.F3

In [0]:
def starting_input():
  index = np.random.randint(0, len(seed_notes) - 32)
  notes = seed_notes[index: index + 32]
  durations = seed_durations[index: index + 32]

  notes_input_sequence, durations_input_sequence = [], []
    
  for n, d in zip(notes,durations):
    note_int = note_to_int[n]
    duration_int = duration_to_int[d]
    
    notes_input_sequence.append(note_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([n, d])
    
      # if n != 'START':
      #   midi_note = note.Note(n)

      #   new_note = np.zeros(128)
      #   new_note[midi_note.pitch.midi] = 1
      #   #overall_preds.append(new_note)

    return notes_input_sequence, durations_input_sequence

In [0]:
def section(notes_input_sequence, durations_input_sequence):
  section_output = []

  for note_index in range(max_extra_notes):
    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    notes_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    
    new_note = np.zeros(128)
    
    for idx, n_i in enumerate(notes_prediction[0]):
        try:
            note_name = int_to_note[idx]
            midi_note = note.Note(note_name)
            new_note[midi_note.pitch.midi] = n_i
            
        except:
            pass
            
    note_sample = sample_with_temp(notes_prediction[0], notes_temp)
    duration_sample = sample_with_temp(durations_prediction[0], duration_temp)

    note_result = int_to_note[note_sample]
    duration_result = int_to_duration[duration_sample]
    
    section_output.append([note_result, duration_result])

    notes_input_sequence.append(note_sample)
    durations_input_sequence.append(duration_sample)
    
    if len(notes_input_sequence) > max_seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
  
#     print(note_result)
#     print(duration_result)
    variation_seed = [notes_input_sequence[-32:], durations_input_sequence[-32:]]
    if note_result == 'START':
        # return notes_input_sequence, durations_input_sequence, prediction_output, variation_seed
        return section_output, variation_seed
  # return notes_input_sequence, durations_input_sequence, prediction_output, variation_seed
  return section_output, variation_seed
#print('Generated sequence of {} notes'.format(len(prediction_output)))
  

In [91]:
structure = ['A', 'A', 'B', 'A']
prediction_output = []
seed_dict = {}
notes_input_sequence, durations_input_sequence = starting_input()

for s in structure:
  if s in seed_dict:
    # notes_input_sequence, durations_input_sequence, section_output, variation_seed = section(variation_seed[0], variation_seed[1])
    section_output, variation_seed = section(seed_dict[s][0], seed_dict[s][1])
  else:
    # notes_input_sequence, durations_input_sequence, section_output, variation_seed = section(notes_input_sequence, durations_input_sequence)
    section_output, variation_seed = section(notes_input_sequence, durations_input_sequence)

  prediction_output += section_output  
  seed_dict[s] = variation_seed

In [92]:
output_folder = os.path.join(run_folder, 'output')

midi_stream = stream.Stream()

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    note_pattern, duration_pattern = pattern
    # pattern is a chord
    if ('.' in note_pattern):
        notes_in_chord = note_pattern.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Piano()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        midi_stream.append(new_chord)
    elif note_pattern == 'rest':
    # pattern is a rest
        new_note = note.Rest()
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Piano()
        midi_stream.append(new_note)
    elif note_pattern != 'START':
    # pattern is a note
        new_note = note.Note(note_pattern)
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Piano()
        midi_stream.append(new_note)



midi_stream = midi_stream.chordify()
timestr = time.strftime("%Y%m%d-%H%M%S")
midi_stream.write('midi', fp=os.path.join(output_folder, 'output-' + timestr + '.mid'))

'/content/gdrive/My Drive/cs230/run/compose/supreme_pop/output/output-20200607-193543.mid'

In [93]:
prediction_output

[['G1.G#1.G2.F3.A3.B3.D4.E4.G4.A4.B4', 0.25],
 ['G#1.E-2.F2.C3.F3.A3.C4.E4.G4.A4', 0.25],
 ['G1.B-1.B1.E2.F#3.G3.B-3.C4.E4.F#4.G4.A4.C5.F#5', 0.25],
 ['F1.B-1.F#2.B2.F3.F#3.A3.C4.F4.G4.A4.B4.C5.F5', 0.5],
 ['E-2.F2.F3.A3.C4.E4.G4.A4.B4.C5', 0.25],
 ['C2', 0.25],
 ['A2.E3.E4.G4.A4.B4', 1.0],
 ['C3.A3.A4.C5', Fraction(1, 6)],
 ['B-1.C2.E2.B2.C3.G4.B4.C5.E5', 1.0],
 ['A2.E3.E4.G4.A4.B4', Fraction(1, 6)],
 ['G4', 0.25],
 ['F#2.C3', Fraction(1, 12)],
 ['C2.C3.F3.A3.C4.E4.B4', Fraction(1, 6)],
 ['G1.B-1.E2.G2.B-3.C4.G4.B4.C5.E5.C6', 0.25],
 ['A2.B2.E3.A3.E4.F#4', Fraction(1, 6)],
 ['G2', Fraction(1, 3)],
 ['E-1.B-1.B2.C#3.E-3.F#3.G3.B-3.E-4.G4.B-4.B4.E-5.G5.G6', Fraction(1, 6)],
 ['G#1.E-2.G2.G3.B3.D4.E4.G4.A4.B4', 0.25],
 ['A2.B-2.A3.C4.E4.F4.G4', Fraction(1, 6)],
 ['B1.C2.E2.F#3.G3.C4.E4.C5.C6', 1.0],
 ['B-1.C2.E2.C3.C4.G4.B4.C6', Fraction(1, 6)],
 ['F#1.C#3.B-3.F#4.B4.C5.D5.F#5.F#6', Fraction(1, 12)],
 ['B-1.C2', 0.25],
 ['F2.B2.B3.C4.D4.E4.F#4.G4.A4', Fraction(1, 12)],
 ['A2.B2.E3.A3.C4.